<a href="https://colab.research.google.com/github/vietsub00/Perceiver_Notebook/blob/main/Perceiver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-addons
!pip install -q --upgrade tf_fourier_features
!pip install einops

     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [ ]:
import numpy as np
from math import pi
import tensorflow as tf
import torch
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from einops import rearrange, repeat
from tf_fourier_features import FourierFeatureProjection

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.19.93.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.19.93.146:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


## Load and preprocess the dataset:

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print(f'x_train shape: {x_train.shape}, y_train shape: {y_train.shape}')
print(f'x_train shape: {x_test.shape}, y_train shape: {y_test.shape}')

x_train = x_train / 255
x_test = x_test / 255

170508288/170498071 [==============================] - 3s 0us/step
x_train shape: (50000, 32, 32, 3), y_train shape: (50000, 1)
x_train shape: (10000, 32, 32, 3), y_train shape: (10000, 1)


## Setup Hyperparameters:

In [ ]:
num_classes = 10
input_shape = x_train.shape[1:]
num_freq_bands = 6          # number of freq bands, with original value (2 * K + 1)
max_freq = 10.0              # maximum frequency, hyperparameter depending on how fine the data is

## Helper Functions:

In [ ]:
#@title Fourier Features in pyTorch { vertical-output: true, display-mode: "form" }
import torch

# data = x_train[:100]
# data = torch.tensor(data)
# print(data.shape)

# def fourier_encode_torch(x, max_freq, num_bands = 4):
#     x = x.unsqueeze(-1)
#     device, dtype, orig_x = x.device, x.dtype, x

#     scales = torch.linspace(1., max_freq / 2, num_bands, device = device, dtype = dtype)
#     scales = scales[(*((None,) * (len(x.shape) - 1)), Ellipsis)]

#     x = x * scales * pi
#     x = torch.cat([x.sin(), x.cos()], dim = -1)
#     x = torch.cat((x, orig_x), dim = -1)

#     return x

# b, *axis, _, = *data.shape,
# # calculate fourier encoded positions in the range of [-1, 1], for all axis
# axis_pos = list(map(lambda size: torch.linspace(-1., 1., steps = size), axis))
# pos = torch.stack(torch.meshgrid(*axis_pos), dim = -1)
# enc_pos = fourier_encode_torch(pos, max_freq, num_freq_bands)
# enc_pos = rearrange(enc_pos, '... n d -> ... (n d)')
# enc_pos = repeat(enc_pos, '... -> b ...', b = b)

# data = torch.cat((data, enc_pos), dim = -1)
# data.shape

torch.arange(11).chunk(6)

(tensor([0, 1]),
 tensor([2, 3]),
 tensor([4, 5]),
 tensor([6, 7]),
 tensor([8, 9]),
 tensor([10]))

In [ ]:
x = pos
num_bands = num_freq_bands

In [ ]:
x = tf.expand_dims(x, -1)
orig_x = x
print(x.shape)
x[10][15]

(32, 32, 2, 1)


<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.35483873],
       [-0.03225809]], dtype=float32)>

In [ ]:
scales = tf.linspace(1., max_freq / 2, num_bands)
scales = scales[(*((None,) * (len(x.shape) - 1)), Ellipsis)]
scales

<tf.Tensor: shape=(1, 1, 1, 6), dtype=float32, numpy=array([[[[1. , 1.8, 2.6, 3.4, 4.2, 5. ]]]], dtype=float32)>

In [ ]:
x = x * scales * pi
x.shape

TensorShape([32, 32, 2, 6])

In [ ]:
x[0][0]

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[ -3.1415927,  -5.6548667,  -8.16814  , -10.681416 , -13.194689 ,
        -15.707964 ],
       [ -3.1415927,  -5.6548667,  -8.16814  , -10.681416 , -13.194689 ,
        -15.707964 ]], dtype=float32)>

In [ ]:
x = tf.concat([tf.sin(x), tf.cos(x)], axis = -1)
x.shape

TensorShape([32, 32, 2, 12])

In [ ]:
x[0][0]

<tf.Tensor: shape=(2, 12), dtype=float32, numpy=
array([[ 8.7422777e-08,  5.8778530e-01, -9.5105666e-01,  9.5105666e-01,
        -5.8778501e-01,  6.7553248e-07, -1.0000000e+00,  8.0901694e-01,
        -3.0901653e-01, -3.0901650e-01,  8.0901718e-01, -1.0000000e+00],
       [ 8.7422777e-08,  5.8778530e-01, -9.5105666e-01,  9.5105666e-01,
        -5.8778501e-01,  6.7553248e-07, -1.0000000e+00,  8.0901694e-01,
        -3.0901653e-01, -3.0901650e-01,  8.0901718e-01, -1.0000000e+00]],
      dtype=float32)>

In [ ]:
def fourier_encode(x, max_freq, num_bands = 4):
  x = tf.expand_dims(x, -1)
  orig_x = x

  scales = tf.linspace(1., max_freq / 2, num_bands)
  scales = scales[(*((None,) * (len(x.shape) - 1)), Ellipsis)]

  x = x * scales * pi
  x = tf.concat([tf.sin(x), tf.cos(x)], axis = -1)
  x = tf.concat((x, orig_x), axis = -1)
  return x

In [ ]:
def GeGLU(x):
  

In [ ]:
data = x_train
b, *axis, _, = *data.shape,

In [ ]:
axis_pos = list(map(lambda size: tf.linspace(-1., 1., num = size), axis))
pos = tf.stack(tf.meshgrid(*axis_pos, indexing='ij'), axis = -1)
print(pos.shape)
pos[16][16]

(32, 32, 2)


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.03225803, 0.03225803], dtype=float32)>

In [ ]:
enc_pos = fourier_encode(pos, max_freq, num_freq_bands)
print(enc_pos.shape)
enc_pos[0][0]

(32, 32, 2, 13)


<tf.Tensor: shape=(2, 13), dtype=float32, numpy=
array([[ 8.7422777e-08,  5.8778530e-01, -9.5105666e-01,  9.5105666e-01,
        -5.8778501e-01,  6.7553248e-07, -1.0000000e+00,  8.0901694e-01,
        -3.0901653e-01, -3.0901650e-01,  8.0901718e-01, -1.0000000e+00,
        -1.0000000e+00],
       [ 8.7422777e-08,  5.8778530e-01, -9.5105666e-01,  9.5105666e-01,
        -5.8778501e-01,  6.7553248e-07, -1.0000000e+00,  8.0901694e-01,
        -3.0901653e-01, -3.0901650e-01,  8.0901718e-01, -1.0000000e+00,
        -1.0000000e+00]], dtype=float32)>

In [ ]:
enc_pos = rearrange(enc_pos, '... n d -> ... (n d)')
print(enc_pos.shape)
enc_pos[0][0]

(32, 32, 26)


<tf.Tensor: shape=(26,), dtype=float32, numpy=
array([ 8.7422777e-08,  5.8778530e-01, -9.5105666e-01,  9.5105666e-01,
       -5.8778501e-01,  6.7553248e-07, -1.0000000e+00,  8.0901694e-01,
       -3.0901653e-01, -3.0901650e-01,  8.0901718e-01, -1.0000000e+00,
       -1.0000000e+00,  8.7422777e-08,  5.8778530e-01, -9.5105666e-01,
        9.5105666e-01, -5.8778501e-01,  6.7553248e-07, -1.0000000e+00,
        8.0901694e-01, -3.0901653e-01, -3.0901650e-01,  8.0901718e-01,
       -1.0000000e+00, -1.0000000e+00], dtype=float32)>

In [ ]:
enc_pos = repeat(enc_pos, '... -> b ...', b = b)
print(enc_pos.shape)
enc_pos[0][0][0]

(50000, 32, 32, 26)


<tf.Tensor: shape=(26,), dtype=float32, numpy=
array([ 8.7422777e-08,  5.8778530e-01, -9.5105666e-01,  9.5105666e-01,
       -5.8778501e-01,  6.7553248e-07, -1.0000000e+00,  8.0901694e-01,
       -3.0901653e-01, -3.0901650e-01,  8.0901718e-01, -1.0000000e+00,
       -1.0000000e+00,  8.7422777e-08,  5.8778530e-01, -9.5105666e-01,
        9.5105666e-01, -5.8778501e-01,  6.7553248e-07, -1.0000000e+00,
        8.0901694e-01, -3.0901653e-01, -3.0901650e-01,  8.0901718e-01,
       -1.0000000e+00, -1.0000000e+00], dtype=float32)>

In [ ]:
data = tf.concat((data, enc_pos), axis = -1)
print(data.shape)

(50000, 32, 32, 29)


In [ ]:
data = x_train
b, *axis, _, = *data.shape,

axis_pos = list(map(lambda size: tf.linspace(-1., 1., num = size), axis))
pos = tf.stack(tf.meshgrid(*axis_pos, indexing='ij'), axis = -1)

enc_pos = fourier_encode(pos, max_freq, num_freq_bands)

enc_pos = rearrange(enc_pos, '... n d -> ... (n d)')
enc_pos = repeat(enc_pos, '... -> b ...', b = b)

data = tf.concat((data, enc_pos), axis = -1)
input_shape = data.shape[1:]
data.shape

In [ ]:
def create_model():
  input = layers.Input(input_shape)
  # x = layers.Dropout(0.3)(x)
  #x = layers.Dense(512, activation='relu')(x)
  # x = layers.Flatten()(x)
  #x = layers.Dense(2048, activation='relu')(x)
  # x = layers.Dense(1024, activation='relu')(x)
  # x = layers.Dropout(0.5)(x)
  # x = layers.Dense(512, activation='relu')(x)
  # x = layers.Dropout(0.5)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(256, activation='relu')(x)
  x = layers.Dense(64, activation='relu')(x)
  output = layers.Dense(num_classes, activation='softmax')(x)

  model = keras.Model(input, output)

  return model

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='Adam', 
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=[keras.metrics.SparseCategoricalAccuracy(name='acc')])
  model.summary()

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=4, factor=0.5)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [ ]:
with strategy.scope():
  history = model.fit(x=data,
                      y=y_train,
                      batch_size=64,
                      epochs=100, 
                      callbacks=[reduce_lr, early_stopping],
                      validation_split=0.2
                      )